# 모듈 임포트

In [89]:
import sys,os
import importlib
current_directory_ipynb = os.getcwd()
parent_directory = os.path.dirname(current_directory_ipynb) # 현재 스크립트 파일의 디렉토리 경로 (상위 디렉토리)를 계산
sys.path.append(parent_directory)
import path_ipynb
# importlib.reload(path_ipynb)
import eKoNL as ek
importlib.reload(ek)
import LDA
importlib.reload(LDA)
import pandas as pd
import mecab_pro as mp
importlib.reload(mp)

# import Lasso
# importlib.reload(Lasso)
data_path = parent_directory + "\\data"

# current_path = os.getcwd() #현 디렉토리
# parent_path = os.path.dirname(current_path) #project_stock_volume
# sys.path.append(parent_path)
# import path


# from text_preprocess import eKoNL as ek
# from LDA_process import LDA
# importlib.reload(ek)
# importlib.reload(LDA)

# LDA진행

## 데이터 불러오기

In [41]:
def report_preprocess(df):
    df.rename(columns={'Report': 'Content'}, inplace=True)

    # 'Content' 칼럼의 개행문자 제거
    df['Content'] = df['Content'].str.replace('\n', '').str.replace('\t', '')

    # 'Date' 칼럼의 데이터 형식을 datetime으로 변환
    df['Date'] = '20' + df['Date']
    df['Date'] = pd.to_datetime(df['Date'])
    return df[['Date','Content','stock']]

In [42]:
#리포트 불러오기
car_report = pd.read_csv(data_path+"\\자동차_combined_report.csv")
car_report['stock'] = '현대차'
semiconduct_report = pd.read_csv(data_path+"\\반도체_combined_report.csv")
semiconduct_report['stock'] = '삼성전자'
bio_report = pd.read_csv(data_path+"\\바이오_combined_report.csv")
bio_report['stock'] = '셀트리온'

#뉴스기사 불러오기
hyundai = pd.read_excel(data_path + "\\현대차_20040101_20221231.xlsx")
hyundai = hyundai[['Date','Content','stock']]
hyundai['Date'] = pd.to_datetime(hyundai['Date'])
samsung = pd.read_excel(data_path + "\\삼성전자_20120101_20221231.xlsx")
samsung = samsung[['Date','Content','stock']]
samsung['Date'] = pd.to_datetime(samsung['Date'])
celltrion = pd.read_excel(data_path + "\\셀트리온_20120101_20221231.xlsx")
celltrion = celltrion[['Date','Content','stock']]
celltrion['Date'] = pd.to_datetime(celltrion['Date'])

# 주가데이터 불러오기
car = pd.read_csv(data_path + "\\car_combined.csv")
car = car.drop(columns=['Unnamed: 0'])
bio = pd.read_csv(data_path + "\\bio_combined.csv")
bio = bio.drop(columns=['Unnamed: 0'])
semiconduct = pd.read_csv(data_path + "\\semiconductor_combined.csv")
semiconduct = semiconduct.drop(columns=['Unnamed: 0'])
filter_condition = (semiconduct['stock'] == '삼성전자') & (semiconduct['Date'] < '2018-05-04')
filtered_indices = semiconduct.index[filter_condition]
semiconduct.loc[filtered_indices, 'Volume'] *= 50

hyundai = hyundai.dropna()
samsung = samsung.dropna()
celltrion = celltrion.dropna()

In [69]:
filter_condition = (semiconduct['stock'] == '삼성전자') & (semiconduct['Date'] < '2018-05-04')
filtered_indices = semiconduct.index[filter_condition]
semiconduct.loc[filtered_indices, 'Volume'] *= 50
semiconduct

,Date,Open,High,Low,Close,Volume,Change,stock
0,2008-01-02,11180,11200,10840,10880,18047200,-0.021583,삼성전자
1,2008-01-03,10760,11060,10640,10920,19346500,0.003676,삼성전자
2,2008-01-04,10920,10960,10760,10780,17997350,-0.012821,삼성전자
3,2008-01-07,10360,10480,10340,10380,39787200,-0.037106,삼성전자
4,2008-01-08,10380,10480,10320,10320,24783700,-0.005780,삼성전자
...,...,...,...,...,...,...,...,...
25052,2023-08-25,1610,1623,1583,1592,1358835,-0.023912,KEC
25053,2023-08-28,1593,1620,1590,1617,812331,0.015704,KEC
25054,2023-08-29,1611,1692,1611,1667,2233987,0.030921,KEC
25055,2023-08-30,1697,1890,1678,1726,13121656,0.035393,KEC


In [43]:
car_report = report_preprocess(car_report)
semiconduct_report = report_preprocess(semiconduct_report)
bio_report = report_preprocess(bio_report)

hyundai_text = pd.concat([hyundai, car_report], axis = 0)
samsung_text = pd.concat([samsung, semiconduct_report], axis = 0)
celltrion_text = pd.concat([celltrion, bio_report], axis = 0)


## 모델 생성

In [45]:
#모델 생성
hyundai_model, hyundai_text, hyundai_corpus = LDA.get_LDA(hyundai_text, 'Content', 50, 42, 10)
samsung_model, samsung_text, samsung_corpus = LDA.get_LDA(samsung_text, 'Content', 50, 42, 10)
celltrion_model, celltrion_text, celltrion_corpus = LDA.get_LDA(celltrion_text, 'Content', 50, 42, 10)

In [46]:

hyundai_topics = hyundai_model.print_topics(num_words=5)
samsung_topics = samsung_model.print_topics(num_words=5)
celltrion_topics = celltrion_model.print_topics(num_words=5)

for topic in hyundai_topics:
    print(topic)

for topic in samsung_topics:
    print(topic)

for topic in celltrion_topics:
    print(topic)


(3, '0.268*"안전" + 0.135*"사고" + 0.079*"화재" + 0.047*"충돌" + 0.037*"감지"')
(23, '0.577*"괴리율" + 0.131*"게재" + 0.100*"협회" + 0.019*"일회성" + 0.015*"양궁"')
(34, '0.196*"출하" + 0.166*"표" + 0.151*"산업" + 0.076*"소매" + 0.064*"증가율"')
(43, '0.044*"지원" + 0.039*"푸르" + 0.035*"사회" + 0.027*"활동" + 0.022*"문화"')
(22, '0.048*"영업이익" + 0.040*"실적" + 0.036*"감소" + 0.029*"환율" + 0.026*"부진"')
(33, '0.121*"관리비" + 0.036*"소수" + 0.033*"부과" + 0.032*"냉각" + 0.029*"초고속"')
(38, '0.095*"파업" + 0.050*"협상" + 0.043*"노사" + 0.036*"임금" + 0.022*"합의"')
(11, '0.121*"주가" + 0.114*"시가총액" + 0.047*"상장" + 0.034*"하락" + 0.034*"상승"')
(14, '0.137*"수소" + 0.059*"차" + 0.023*"인프라" + 0.023*"계획" + 0.019*"트럭"')
(20, '0.021*"않" + 0.020*"보" + 0.019*"크" + 0.018*"없" + 0.011*"가장"')
(46, '0.195*"모델" + 0.110*"출시" + 0.040*"판매" + 0.039*"신차" + 0.029*"신형"')
(32, '0.101*"기업" + 0.092*"유형자산" + 0.046*"법인세" + 0.020*"인" + 0.020*"금융"')
(19, '0.041*"주행" + 0.033*"적용" + 0.026*"시스템" + 0.021*"연비" + 0.019*"모델"')
(41, '0.074*"사업" + 0.032*"축" + 0.028*"규모" + 0.025*"설립" + 0.021*"투자"')
(

# LASSO 회귀 위한 데이터 전처리

In [81]:
def process_stock_data(df):
    unique_stocks = df['stock'].unique()  # 주식명(stock)의 고유한 값들을 가져옴    
    processed_dfs = []  # 처리된 데이터프레임들을 저장할 리스트
    
    for stock_name in unique_stocks:
        stock_df = df[df['stock'] == stock_name].copy()  # 주식명에 해당하는 부분을 선택하여 복사
        
        ### 주가 상승 예측 모델 생성 위한 1일 1주일 1달 3달 6달 1년 뒤 상승여부 컬럼 만들기(종가 기준)
        stock_df['1day_updown'] = (stock_df['Close'].shift(-1) > stock_df['Close']).astype(int)
        stock_df['1week_updown'] = (stock_df['Close'].shift(-5) > stock_df['Close']).astype(int)
        stock_df['1month_updown'] = (stock_df['Close'].shift(-20) > stock_df['Close']).astype(int)  # 가정: 1달은 20 영업일로 계산
        stock_df['3month_updown'] = (stock_df['Close'].shift(-60) > stock_df['Close']).astype(int)  # 가정: 3달은 60 영업일로 계산
        stock_df['6month_updown'] = (stock_df['Close'].shift(-120) > stock_df['Close']).astype(int)  # 가정: 6달은 120 영업일로 계산
        stock_df['1year_updown'] = (stock_df['Close'].shift(-240) > stock_df['Close']).astype(int)
        ### 논문의 거래량 조정 방법인 100거래일 최저 거래량 빼주기(0이 안 나오게끔 0.9를 곱해서 보정 해주었음)
        # minus = stock_df['Volume'].rolling(window=100).min()
        # minus.fillna(method='bfill', inplace=True) # 뒤의 값을 통해서 결측값을 채우기(첫 99일 동안은 nan이 나오기 때문)
        # minus = minus * 0.1
        # stock_df['Target'] = stock_df['Volume'] - minus
        
        ## 주말 공휴일 등, 거래되지 않은 날의 주가 및 거래 데이터를 앞의 날의 정보로 채워 넣기
        stock_df['Date'] = pd.to_datetime(stock_df['Date'])
        all_dates = pd.date_range(start=min(stock_df['Date']), end=max(stock_df['Date']))  # 모든 날짜 생성
        stock_df = stock_df.set_index('Date').reindex(all_dates)  # 날짜 인덱스 재설정 및 빈 날짜 생성
        stock_df = stock_df.fillna(method='ffill')  # 빈 값을 앞쪽으로 채우기
                  
       
        processed_dfs.append(stock_df)
    
    # 모든 데이터프레임을 병합
    result_df = pd.concat(processed_dfs, axis=0)
    result_df = result_df.reset_index()
    result_df = result_df.rename(columns={'index': 'Date'})

    # 예측을 위한 수정 거래량 구하는 코드
    

    return result_df

filled_df = process_stock_data(semiconduct)
filled_df[:30]

,Date,Open,High,Low,Close,Volume,Change,stock,1day_updown,1week_updown,1month_updown,3month_updown,6month_updown,1year_updown,Target
0,2008-01-02,11180.0,11200.0,10840.0,10880.0,18047200.0,-0.021583,삼성전자,1.0,0.0,1.0,1.0,1.0,0.0,16988035.0
1,2008-01-03,10760.0,11060.0,10640.0,10920.0,19346500.0,0.003676,삼성전자,0.0,0.0,1.0,1.0,1.0,0.0,18287335.0
2,2008-01-04,10920.0,10960.0,10760.0,10780.0,17997350.0,-0.012821,삼성전자,0.0,0.0,1.0,1.0,1.0,0.0,16938185.0
3,2008-01-05,10920.0,10960.0,10760.0,10780.0,17997350.0,-0.012821,삼성전자,0.0,0.0,1.0,1.0,1.0,0.0,16938185.0
4,2008-01-06,10920.0,10960.0,10760.0,10780.0,17997350.0,-0.012821,삼성전자,0.0,0.0,1.0,1.0,1.0,0.0,16938185.0
5,2008-01-07,10360.0,10480.0,10340.0,10380.0,39787200.0,-0.037106,삼성전자,0.0,1.0,1.0,1.0,1.0,0.0,38728035.0
6,2008-01-08,10380.0,10480.0,10320.0,10320.0,24783700.0,-0.005780,삼성전자,1.0,1.0,1.0,1.0,1.0,0.0,23724535.0
7,2008-01-09,10340.0,10580.0,10260.0,10500.0,23865450.0,0.017442,삼성전자,0.0,1.0,1.0,1.0,1.0,0.0,22806285.0
8,2008-01-10,10439.0,10580.0,10320.0,10320.0,31551350.0,-0.017143,삼성전자,0.0,1.0,1.0,1.0,1.0,0.0,30492185.0
9,2008-01-11,10360.0,10520.0,10220.0,10320.0,23425150.0,0.000000,삼성전자,1.0,1.0,1.0,1.0,1.0,0.0,22365985.0


In [92]:
semiconduct

,Date,Open,High,Low,Close,Volume,Change,stock
0,2008-01-02,11180,11200,10840,10880,18047200,-0.021583,삼성전자
1,2008-01-03,10760,11060,10640,10920,19346500,0.003676,삼성전자
2,2008-01-04,10920,10960,10760,10780,17997350,-0.012821,삼성전자
3,2008-01-07,10360,10480,10340,10380,39787200,-0.037106,삼성전자
4,2008-01-08,10380,10480,10320,10320,24783700,-0.005780,삼성전자
...,...,...,...,...,...,...,...,...
25052,2023-08-25,1610,1623,1583,1592,1358835,-0.023912,KEC
25053,2023-08-28,1593,1620,1590,1617,812331,0.015704,KEC
25054,2023-08-29,1611,1692,1611,1667,2233987,0.030921,KEC
25055,2023-08-30,1697,1890,1678,1726,13121656,0.035393,KEC


In [98]:
import stockdata_process
importlib.reload(stockdata_process)
hyundai_model_df = stockdata_process.get_analyze_df(hyundai_text, car)
samsung_model_df = stockdata_process.get_analyze_df(samsung_text, semiconduct)
celltrion_model_df = stockdata_process.get_analyze_df(celltrion_text, bio)

In [101]:
semiconduct[semiconduct['stock'] == '삼성전자']

,Date,Open,High,Low,Close,Volume,Change,stock
0,2008-01-02,11180,11200,10840,10880,18047200,-0.021583,삼성전자
1,2008-01-03,10760,11060,10640,10920,19346500,0.003676,삼성전자
2,2008-01-04,10920,10960,10760,10780,17997350,-0.012821,삼성전자
3,2008-01-07,10360,10480,10340,10380,39787200,-0.037106,삼성전자
4,2008-01-08,10380,10480,10320,10320,24783700,-0.005780,삼성전자
...,...,...,...,...,...,...,...,...
3870,2023-08-25,67100,67400,66900,67100,7032462,-0.016129,삼성전자
3871,2023-08-28,66800,67000,66500,66800,5824628,-0.004471,삼성전자
3872,2023-08-29,66900,67200,66600,66800,9114352,0.000000,삼성전자
3873,2023-08-30,67300,67700,67100,67100,9181223,0.004491,삼성전자


In [99]:
samsung_model_df['Target']

0        10944900.0
1        10944900.0
2        10944900.0
3        10944900.0
4        10944900.0
            ...    
70515    13174578.0
70516    13174578.0
70517    15044463.0
70518     9181223.0
70519    15964630.0
Name: Target, Length: 70520, dtype: float64

In [96]:
print(filled_df['Target'][:30])
print('===========================================')
samsung_model_df['Target'] = filled_df[filled_df['stock']=='삼성전자']['Target']

0     16988035.0
1     18287335.0
2     16938185.0
3     16938185.0
4     16938185.0
5     38728035.0
6     23724535.0
7     22806285.0
8     30492185.0
9     22365985.0
10    22365985.0
11    22365985.0
12    21107835.0
13    39224785.0
14    48388935.0
15    41154735.0
16    34527235.0
17    34527235.0
18    34527235.0
19    24025885.0
20    42154485.0
21    35244685.0
22    26842835.0
23    25048335.0
24    25048335.0
25    25048335.0
26    26154035.0
27    20412085.0
28    34710235.0
29    45707935.0
Name: Target, dtype: float64


In [57]:
#거래량 정규화 논문 100거래일 최저거래량으로 빼줘서 0일 경우에는 보정
#일자별로 num_w를 주제별로 다 더하고 그러면 20개컬럼(input)을 1개 거래량 컬럼(output) -> model
# peakday 단어수들 20개컬럼 -> 모델 넣어요 -> output 일자별 거래량이랑 output비교 output이 일자별 거래량의 30%이상일 경우 성공으로 간주
# 성공으로 간주된 날이 토요타의 경우 70%, 꽤많은 기업들이 8-90%

lasso_df = pd.merge(hyundai, filled_df, how='left', on=['stock','Date'])

In [74]:
def make_lasso_df(df):
    df = df[['Date','lda_topics','num_w','Target']]
    result_df = df.pivot_table(index=['Date', 'Target'], columns='lda_topics', values='num_w', aggfunc='sum').reset_index()
    result_df = result_df.fillna(0)
    return result_df
hyundai_lasso = make_lasso_df(hyundai_model_df)
samsung_lasso = make_lasso_df(samsung_model_df)
celltrion_lasso = make_lasso_df(celltrion_model_df)
# lasso_hyundai = hyundai_model_df[['Date','lda_topics','num_w','Target']]
# # result_df = df.groupby(['Date', 'lda_topics']).agg({'num_w': 'sum'}).reset_index()
# # result_df
# result_df = df.pivot_table(index=['Date', 'Target'], columns='lda_topics', values='num_w', aggfunc='sum').reset_index()

# # 컬럼명 변경 (lda_topic의 숫자를 직접 명시하려면 컬럼명을 수동으로 수정)
# # result_df.columns = ['Date', 'Target'] + ['lda_topic_' + str(col) for col in result_df.columns[2:]]

# # NaN 값 0으로 채우기
# result_df = result_df.fillna(0)

# # 결과 출력
# result_df.columns

In [66]:
import Lasso
hyundai_analyze = Lasso.lasso(hyundai_lasso, 'hyundai')
samsung_analyze = Lasso.lasso(samsung_lasso, 'samsung')
celltrion_analyze = Lasso.lasso(celltrion_lasso, 'celltrion')

ModuleNotFoundError: No module named 'Lasso'

In [79]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, make_scorer

# 데이터프레임 생성 (앞서 생성한 데이터프레임을 사용)
# 이 데이터프레임에는 Date, lda_topic_0, lda_topic_1, ..., lda_topic_19, target 컬럼이 있어야 합니다.

# Feature(독립변수)와 Target(종속변수)를 나눕니다.
X = result_df.drop(columns=['Date', 'Target'])  # Feature로 lda_topic 컬럼들을 사용
y = result_df['Target']  # Target으로 'target' 컬럼을 사용

# Lasso 회귀 모델 생성
lasso_model = Lasso(alpha=1.0)  # alpha는 L1 규제 강도 조절

# 100회의 10겹 교차 검증 수행
num_cv_iterations = 100
cv_object = KFold(n_splits=10, shuffle=True, random_state=42)

# 교차 검증을 통한 모델 평가 (MSE를 사용하여)
scorer = make_scorer(mean_squared_error, greater_is_better=False)
mse_scores = cross_val_score(lasso_model, X, y, cv=cv_object, scoring='neg_mean_squared_error', n_jobs=-1)
mse_scores = -mse_scores  # 스코어 값은 양수로 변환

# 각 교차 검증에서의 MSE 평균과 표준편차 출력
print(f"Mean MSE: {mse_scores.mean()}")
print(f"Std MSE: {mse_scores.std()}")

UnicodeEncodeError: 'ascii' codec can't encode characters in position 18-20: ordinal not in range(128)

In [75]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# 데이터프레임 생성 (앞서 생성한 데이터프레임을 사용)
# 이 데이터프레임에는 Date, lda_topic_0, lda_topic_1, ..., lda_topic_19, target 컬럼이 있어야 합니다.

# Feature(독립변수)와 Target(종속변수)를 나눕니다.
X = samsung_lasso.drop(columns=['Date', 'Target'])  # Feature로 lda_topic 컬럼들을 사용
y = samsung_lasso['Target']  # Target으로 'target' 컬럼을 사용

# 데이터를 학습용과 테스트용으로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lasso 회귀 모델 생성 및 학습
lasso_model = Lasso(alpha=1.0)  # alpha는 L1 규제 강도 조절
lasso_model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = lasso_model.predict(X_test)

# 모델 평가 (예시로 MSE와 R-squared 사용)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

Mean Squared Error (MSE): 45923845297879.07
R-squared (R2): 0.03654097750194807


In [76]:
from sklearn.metrics import accuracy_score

top_5_percent = samsung_lasso[samsung_lasso['Target'] >= samsung_lasso['Target'].quantile(0.95)]
top_5_percent
X = top_5_percent.iloc[:, 2:52]
top_5_percent['Target_predict'] = lasso_model.predict(X)
threshold = 0.3
top_5_percent['Success'] = top_5_percent['Target_predict'] >= (top_5_percent['Target'] * (threshold))

# 정확도 계산
accuracy = accuracy_score(top_5_percent['Success'], [True] * len(top_5_percent))
print(accuracy)

0.8883928571428571


C:\Users\서울디지털인재개발원\AppData\Local\Temp\ipykernel_19740\928692301.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_5_percent['Target_predict'] = lasso_model.predict(X)
C:\Users\서울디지털인재개발원\AppData\Local\Temp\ipykernel_19740\928692301.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_5_percent['Success'] = top_5_percent['Target_predict'] >= (top_5_percent['Target'] * (threshold))


In [61]:
accuracy

0.5491071428571429

In [63]:
samsung_lasso

lda_topics,Date,Target,0,1,2,3,4,5,6,7,...,40,41,42,43,44,45,46,47,48,49
0,2008-01-18,690544.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2008-01-21,480517.7,0.0,0.0,737.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2008-01-24,536856.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2008-01-25,500966.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2008-02-01,545858.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4474,2023-08-14,8535140.9,0.0,0.0,611.0,0.0,0.0,0.0,0.0,0.0,...,0.0,390.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4475,2023-08-16,12357375.9,0.0,0.0,2732.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,440.0,0.0,0.0,0.0
4476,2023-08-24,14227260.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4477,2023-08-30,8598760.2,0.0,0.0,3945.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
top_5_percent

lda_topics,Date,Target,0,1,2,3,4,5,6,7,...,42,43,44,45,46,47,48,49,Target_predict,Success
2700,2018-05-04,39565391.0,0.0,0.0,158.0,0.0,0.0,0.0,0.0,605.0,...,172.0,0.0,0.0,0.0,0.0,502.0,0.0,0.0,6.476182e+06,False
2701,2018-05-06,39565391.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,165.0,179.0,0.0,0.0,6.310461e+06,False
2702,2018-05-07,39565391.0,0.0,0.0,0.0,0.0,0.0,85.0,0.0,182.0,...,0.0,0.0,0.0,0.0,0.0,165.0,0.0,0.0,5.312000e+06,False
2703,2018-05-08,23104720.0,0.0,0.0,0.0,0.0,0.0,104.0,304.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.067938e+06,True
2726,2018-05-31,63491109.0,0.0,692.0,0.0,0.0,0.0,0.0,0.0,0.0,...,676.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.038859e+06,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4214,2022-07-01,24159808.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,334.0,0.0,0.0,0.0,0.0,0.0,566.0,0.0,7.184860e+06,False
4215,2022-07-02,24159808.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.851281e+06,False
4216,2022-07-03,24159808.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.796214e+06,False
4448,2023-05-30,26710392.4,0.0,0.0,482.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.601584e+06,False
